In [20]:
#Author: Kazushi Suzuki and Yoshihiro Shimizu
#Revised: 2020-09-01 
#Reference: 
#Purpose: Analysis of next generation sequencing data for an Affibody library
#Data
#Raw data was deposited into the Squence Read Archive (SRA) of the U.S. National Library of Medicine from the National Center for Biotechnoly Information (NCBI) 
#Accession numbers: 
#
#
#
#

In [23]:
### basic 
import re
import regex
import numpy as np
import pandas as pd
import os

### handling of sequence data
from Bio.Seq import Seq                         
from Bio import SeqIO
from Bio.Alphabet import IUPAC     

In [25]:
%%time
# input-file and output-file location
indir="./raw_data/"
outdir="./"

### sequences which identify region of interest 
Start = "GTTGACAACA"
End = "CCTTCTCAGTC"

filenames=os.listdir(indir)
for i in range(len(filenames)):
    Affibody_protein={}
    if (filenames[i][-5:]=='fastq'):
        for record in SeqIO.parse(indir+filenames[i], "fastq") :
            recordqual=[x>21 for x in record.letter_annotations['phred_quality']] 
             #only process reads that have more than half of basecalls >Q21
            if  (float(sum(recordqual))/float(len(recordqual))>=.5): 
                recordseq="".join([y if x else 'N' for (x,y) in zip(recordqual, record.seq)]) 
                if (recordseq.find(Start)>0) and (recordseq.find(End)>0):
                    Start_index = recordseq.find(Start) 
                    End_index = recordseq.find(End) + len(End)
                    Affibody_cds = Seq(recordseq[Start_index:End_index], IUPAC.ambiguous_dna)
                    #translates DNA 
                    if (len(Affibody_cds) == 122) :
                        Affibody_protein[record.id] = Affibody_cds.translate()
        
    Counts=pd.DataFrame(Affibody_protein.items(), columns=['ID', 'CDRs'])
    Counts=Counts.groupby('CDRs').count().sort_values('ID', ascending=False)
    Counts.to_csv(outdir+filenames[i].strip('.fastq')+'.csv') 

CPU times: user 12min 5s, sys: 6.97 s, total: 12min 12s
Wall time: 12min 12s


In [27]:
df_FR6 = pd.read_csv('i-FR6_S10_L001_R1_001.csv', header=0)
df_R6 = pd.read_csv('i-R6_S11_L001_R1_001.csv', header=0)

In [28]:
Df = pd.merge(df_R6, df_FR6, on='CDRs', how='outer')
Df = Df.query('not CDRs.str.contains("X")', engine='python')
Df = Df.fillna(0)
Df.columns = ['CDRs', 'R6', 'FR6']
Df = Df[Df['FR6']==0]
Df.to_csv(outdir+"Summary"+'.csv') 